In [122]:
from nornir import InitNornir
from nornir.core.inventory import Host
import json

nr = InitNornir(config_file="config.yaml")

# prints the schema designed for hosts file and groups file
print(json.dumps(Host.schema(), indent=4))

# prints hosts
print("####All hosts seen in inventory####")
print(nr.inventory.hosts.keys())

# prints groups
print("####Printing groups####")
print(nr.inventory.groups)

# inheritence
lab_dcs_1 = nr.inventory.hosts['lab-dcs-1']

#Printing vlans
print("####Printing vlans####")
print(lab_dcs_1['vlans'])

{
    "name": "str",
    "connection_options": {
        "$connection_type": {
            "extras": {
                "$key": "$value"
            },
            "hostname": "str",
            "port": "int",
            "username": "str",
            "password": "str",
            "platform": "str"
        }
    },
    "groups": [
        "$group_name"
    ],
    "data": {
        "$key": "$value"
    },
    "hostname": "str",
    "port": "int",
    "username": "str",
    "password": "str",
    "platform": "str"
}
####All hosts seen in inventory####
dict_keys(['lab-dcs-1', 'lab-dcs-2', 'lab-mlx-1', 'lab-qfx-1', 'lab-qfx-2', 'lab-vdx', 'lab-slx-1', 'lab-slx-2'])
####Printing groups####
{'eos': Group: eos, 'ironware': Group: ironware, 'junos': Group: junos, 'nos': Group: nos, 'slxos': Group: slxos}
####Printing vlans####
[{'name': 'TEST-VLAN-10', 'vlan_id': 10}, {'name': 'TEST-VLAN-15', 'vlan_id': 15}, {'name': 'TEST-VLAN-20', 'vlan_id': 20}, {'name': 'TEST-VLAN-30', 'vlan_id': 30}]


In [114]:
# printing username
print(lab_dcs_1)

KeyError: 'username'

In [92]:
# printing domain from global
print(lab_vdx_1)

lab-dcs-1


In [103]:
from nornir.core.task import Task, Result
from nornir_utils.plugins.functions import print_result

# setting function to print hello_world
def hello_world(task: Task) -> Result:
    return Result(
        host=task.host,
        result=f"{task.host} says hello world!"
    )
result = nr.run(task=hello_world)

print_result(result)

hello_world*********************************************************************


In [105]:
# Setting function to take in text
def say(task: Task, text: str) -> Result:
    return Result(
        host=task.host,
        result=f"{task.host} says {text}"
    )
result = nr.run(
    name="Saying goodbye in a friendly manner",
    task=say,
    text="goodbye!")

print_result(result)

Saying goodbye in a friendly manner*********************************************
* lab-dcs-1 ** changed : False *************************************************
vvvv Saying goodbye in a friendly manner ** changed : False vvvvvvvvvvvvvvvvvvvv INFO
lab-dcs-1 says goodbye!
^^^^ END Saying goodbye in a friendly manner ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* lab-dcs-2 ** changed : False *************************************************
vvvv Saying goodbye in a friendly manner ** changed : False vvvvvvvvvvvvvvvvvvvv INFO
lab-dcs-2 says goodbye!
^^^^ END Saying goodbye in a friendly manner ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* lab-mlx-1 ** changed : False *************************************************
vvvv Saying goodbye in a friendly manner ** changed : False vvvvvvvvvvvvvvvvvvvv INFO
lab-mlx-1 says goodbye!
^^^^ END Saying goodbye in a friendly manner ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* lab-qfx-1 ** changed : False *************************************************
vvvv Saying goodbye in

In [107]:
def count(task: Task, number: int) -> Result:
    return Result(
        host=task.host,
        result=f"{[n for n in range(0, number)]}"
    )

def greet_and_count(task: Task, number: int) -> Result:
    task.run(
        name="Counting beans",
        task=count,
        number=number,
    )
    task.run(
        name="We should say bye too",
        task=say,
        text="bye!",
    )

    # let's inform if we counted even or odd times
    even_or_odds = "even" if number % 2 == 1 else "odd"
    return Result(
        host=task.host,
        result=f"{task.host} counted {even_or_odds} times!",
    )

result = nr.run(
    name="Counting to 5 while being very polite",
    task=greet_and_count,
    number=5,
)
print_result(result)

Counting to 5 while being very polite*******************************************
* lab-dcs-1 ** changed : False *************************************************
vvvv Counting to 5 while being very polite ** changed : False vvvvvvvvvvvvvvvvvv INFO
lab-dcs-1 counted even times!
---- Counting beans ** changed : False ----------------------------------------- INFO
[0, 1, 2, 3, 4]
---- We should say bye too ** changed : False ---------------------------------- INFO
lab-dcs-1 says bye!
^^^^ END Counting to 5 while being very polite ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* lab-dcs-2 ** changed : False *************************************************
vvvv Counting to 5 while being very polite ** changed : False vvvvvvvvvvvvvvvvvv INFO
lab-dcs-2 counted even times!
---- Counting beans ** changed : False ----------------------------------------- INFO
[0, 1, 2, 3, 4]
---- We should say bye too ** changed : False ---------------------------------- INFO
lab-dcs-2 says bye!
^^^^ END Counting to 5 while

In [99]:
# importing necessary modules to filter and send show command
from nornir.core.filter import F
from nornir_napalm.plugins.tasks import napalm_cli

devices_junos_eos = nr.filter(F(platform='junos') | F(platform='eos'))

# run the following command
result = devices_junos_eos.run(napalm_cli, commands=['show version'])

print_result(result)

napalm_cli**********************************************************************


In [93]:
from nornir_napalm.plugins.tasks import napalm_get

# Using napalm_get to retrieve facts data
result = devices_junos_eos.run(napalm_get, getters=['arp_table'])

print_result(result)

napalm_get**********************************************************************
* lab-dcs-1 ** changed : False *************************************************
vvvv napalm_get ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
{ 'arp_table': [ { 'age': 0.0,
                   'interface': 'Management1',
                   'ip': '10.198.64.1',
                   'mac': '00:31:46:15:D7:F0'}]}
^^^^ END napalm_get ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* lab-dcs-2 ** changed : False *************************************************
vvvv napalm_get ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
{ 'arp_table': [ { 'age': 0.0,
                   'interface': 'Management1',
                   'ip': '10.198.64.1',
                   'mac': '00:31:46:15:D7:F0'}]}
^^^^ END napalm_get ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* lab-qfx-1 ** changed : False *************************************************
vvvv nap

In [81]:
from nornir import InitNornir

# Using Ansible Inventory plugin
nr = InitNornir(config_file="config.yaml")
# print plugin
print(nr.config.inventory.plugin)
# print all hosts
print(nr.inventory.hosts)

AnsibleInventory
{'ztp': Host: ztp, 'lab-dcs-1': Host: lab-dcs-1, 'lab-dcs-2': Host: lab-dcs-2, 'lab-mlx-1': Host: lab-mlx-1, 'lab-qfx-1': Host: lab-qfx-1, 'lab-qfx-2': Host: lab-qfx-2, 'lab-vdx': Host: lab-vdx, 'lab-slx-1': Host: lab-slx-1, 'lab-slx-2': Host: lab-slx-2}


In [113]:
from nornir import InitNornir
from nornir.core.filter import F
from nornir.core.task import Task, Result
from nornir_utils.plugins.functions import print_result
from nornir_napalm.plugins.tasks import napalm_get

# Setting function to take in text
def get_facts(task: Task) -> Result:
    facts = task.run(
        task=napalm_get, getters=['facts']
    )
    return Result(
        host=task.host,
        result=facts
    )

nr = InitNornir(config_file="config.yaml")

# run only against junos or eos
devices_junos_eos = nr.filter(F(platform="junos") | F(platform="eos"))

result = devices_junos_eos.run(
    name="Gathering users from device",
    task=get_facts
)

print_result(result)

Gathering users from device*****************************************************
